In [1]:
import numpy as np
import random

## 9.2

In [2]:
gamma = 3/4
p_pie = np.array([[1/3, 2/3, 0], [2/3, 1/3, 0], [0, 2/3, 1/3]])
r = np.array([24, -6, 12])
v_pie = np.dot(np.linalg.inv(np.identity(3) - gamma*p_pie),r)
v_pie

array([48., 24., 32.])

In [3]:
p_a_down = np.array([[1/3, 0, 2/3], [2/3, 1/3, 0], [0, 2/3, 1/3]])
p_a_up = np.array([[1/3, 2/3, 0], [0, 1/3, 2/3], [2/3, 0, 1/3]])

In [4]:
for s in range(3):
    print('s = {}'.format(s+1))
    down = np.sum(np.multiply(p_a_down[s], v_pie))
    up = np.sum(np.multiply(p_a_up[s], v_pie))
    print('a = down\t', down)
    print('a = up\t\t', up)
    if max(down, up) == down:
        print('pi_prime({}) = down'.format(s+1))
    else:
        print('pi_prime({}) = up'.format(s+1))
    print()

s = 1
a = down	 37.33333333333333
a = up		 32.0
pi_prime(1) = down

s = 2
a = down	 39.99999999999999
a = up		 29.333333333333336
pi_prime(2) = down

s = 3
a = down	 26.66666666666667
a = up		 42.66666666666666
pi_prime(3) = up



In [6]:
gamma = 2/3
p_pie = np.array([[3/4, 1/4], [1/4, 3/4]])
r = np.array([-4, 8])
v_pie = np.dot(np.linalg.inv(np.identity(2) - gamma*p_pie),r)
v_pie

array([-3., 15.])

In [7]:
p_a_down = np.array([[3/4, 1/4], [1/4, 3/4]])
p_a_up = np.array([[2/4, 2/4], [2/4, 2/4]])

In [9]:
for s in range(2):
    print('s = {}'.format(s+1))
    down = np.sum(np.multiply(p_a_down[s], v_pie))
    up = np.sum(np.multiply(p_a_up[s], v_pie))
    print('a = down\t', down)
    print('a = up\t\t', up)
    if max(down, up) == down:
        print('pi_prime({}) = down'.format(s+1))
    else:
        print('pi_prime({}) = up'.format(s+1))
    print()

s = 1
a = down	 1.5
a = up		 6.0
pi_prime(1) = up

s = 2
a = down	 10.5
a = up		 6.0
pi_prime(2) = down



## 9.4

We are given a maze with 81 states (81 squares), 4 actions (up, down, left, right) and discount factor of 0.99. 

### (a) Policy iteration

In [338]:
# constants
gamma = 0.99
r = np.loadtxt('rewards.txt')
p_a1 = np.loadtxt('prob_a1.txt')
s_transitions = p_a1[:,:2].astype('int')
p_a2 = np.loadtxt('prob_a2.txt')
p_a3 = np.loadtxt('prob_a3.txt')
p_a4 = np.loadtxt('prob_a4.txt')
p = np.stack((p_a1[:,2], p_a2[:,2], p_a3[:,2], p_a4[:,2]))
s_start = list(s_transitions[:,0])
N = len(r)

In [339]:
# 1. Initialize policies randomly
pi_init = [random.randrange(4) for i in range(N)]

In [340]:
# 2. Evaluate current policy
def v_pi(pi):
    
    # Create probability matrix
    p_pi = np.zeros((N, N))
    
    # for each s
    for i in range(N):
        # probability of transition to s'
        s = i+1    # value of state
        
        # get possible next states given current state
        next_s_i = [j for j, x in enumerate(s_start) if x == s]
        next_s_val = [(s_transitions[j,1]) for j in next_s_i]
        next_s_nonzero_probs = [p[pi[i]][j] for j in next_s_i]
        
        probs_i = [0]*N
        for v,pr in zip(next_s_val, next_s_nonzero_probs):
            probs_i[v-1] = pr
        
        p_pi[i, :] = np.array(probs_i) # for all next states
        
    
    v = np.dot(np.linalg.inv(np.identity(N) - gamma*p_pi),r)
    
    return v

In [341]:
# 3. Update policy
def improve_policy(v):
    opt = []
    
    # for all states
    for i in range(N):
        s = i+1    # value of state
    
        if v[i] != 0:
            # get possible next states given current state
            next_s_i = [i for i, x in enumerate(s_start) if x == s] 
            
            # use maximum value across all actions
            max_val = 0
            a_opt = 0
            for a in range(4):            
                # value iteration update
                val = r[i] + gamma*sum([p[a][j]*v[s_transitions[j,1] - 1] for j in next_s_i])
                
                # compare to current max value
                if val > max_val:
                    max_val = val
                    a_opt = a
            
            # update to maximum value
            opt.append(a_opt)
        else:
            #opt.append('')
            opt.append(0)
            
    return opt

In [342]:
pi_curr = pi_init.copy()
pi_next = [0]*N
no_change = pi_next == pi_curr
#for i in range(15):
while not no_change:
    # compute value function
    v_curr = v_pi(pi_curr)
    
    # improve policy
    pi_next = improve_policy(v_curr)
    
    # check if change
    no_change = pi_next == pi_curr
    
    # update pi
    pi_curr = pi_next

In [343]:
v_curr = v_pi(pi_curr)
np.transpose(np.round(np.array(v_curr),2).reshape((9,9)))

array([[   0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ,    0.  ],
       [   0.  ,   65.77,   67.14,   77.85,   79.84,   72.48, -100.  ,
           0.  ,  100.  ],
       [   0.  ,   55.88, -100.  ,   70.31,   81.34,   83.05,   84.88,
          96.87,   98.72],
       [   0.  ,   54.92,   50.48,   59.67,    0.  ,   80.96,    0.  ,
          97.04,   98.73],
       [  53.51,   54.15,    0.  , -100.  , -100.  ,   61.78, -100.  ,
          88.22,  100.  ],
       [   0.  ,   52.5 ,   43.94,   51.09,   61.01,   71.79,   73.95,
          85.18,   97.57],
       [   0.  ,   43.77, -100.  ,    0.  ,    0.  ,   70.35,    0.  ,
        -100.  ,   88.41],
       [   0.  ,   47.95,   48.77,   58.15,   59.39,   60.17, -100.  ,
           0.  ,  100.  ],
       [   0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ,    0.  ]])

In [329]:
actions = ['W', 'N', 'E', 'S']
opt_moves = []
for i in range(N):
    if v_curr[i] > 0:
        opt_moves.append(actions[pi_curr[i]])
    else:
        opt_moves.append('')
        
np.transpose(np.array(opt_moves).reshape((9,9)))

array([['', '', '', '', '', '', '', '', ''],
       ['', 'E', 'E', 'E', 'S', 'S', '', '', 'W'],
       ['', 'N', '', 'E', 'E', 'E', 'E', 'E', 'N'],
       ['', 'N', 'E', 'N', '', 'N', '', 'E', 'S'],
       ['E', 'N', '', '', '', 'N', '', 'E', 'W'],
       ['', 'N', 'W', 'E', 'E', 'E', 'E', 'E', 'N'],
       ['', 'N', '', '', '', 'N', '', '', 'S'],
       ['', 'E', 'E', 'E', 'E', 'N', '', '', 'W'],
       ['', '', '', '', '', '', '', '', '']], dtype='<U1')

### (b) Value iteration

In [331]:
# constants
gamma = 0.99
r = np.loadtxt('rewards.txt')
p_a1 = np.loadtxt('prob_a1.txt')
s_transitions = p_a1[:,:2].astype('int')
p_a2 = np.loadtxt('prob_a2.txt')
p_a3 = np.loadtxt('prob_a3.txt')
p_a4 = np.loadtxt('prob_a4.txt')
p = np.stack((p_a1[:,2], p_a2[:,2], p_a3[:,2], p_a4[:,2]))
s_start = list(s_transitions[:,0])

In [332]:
# 1. Initialize value function for all states (81 functions) to zero
v_init = [0]*81

In [333]:
def update_v(v):
    v_next = []
    # update all states
    for i in range(81):
        s = i+1    # value of state
        
        # get possible next states given current state
        next_s_i = [i for i, x in enumerate(s_start) if x == s] 
        
        # use maximum value across all actions
        max_val = 0
        for a in range(4):            
            # value iteration update
            val = r[i] + gamma*sum([p[a][j]*v[s_transitions[j,1] - 1] for j in next_s_i])
            
            # compare to current max value
            if val > max_val:
                max_val = val
        
        # update to maximum value
        v_next.append(max_val)
            
    return v_next

In [334]:
# 2. Iterate until convergence

v_curr = v_init.copy()
diff = 1000000
# 10^-5
while diff > 10**-5:
    v_next = update_v(v_curr)
    
    diff = sum([i - j for i,j in zip(v_next, v_curr)])
    
    v_curr = v_next

In [344]:
np.transpose(np.array(v_curr).reshape((9,9)))

array([[   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ],
       [   0.        ,   65.77308407,   67.13647421,   77.84605   ,
          79.84451583,   72.47511769, -100.        ,    0.        ,
         100.        ],
       [   0.        ,   55.88294346, -100.        ,   70.30818136,
          81.34440225,   83.04847989,   84.88054612,   96.87232244,
          98.71875987],
       [   0.        ,   54.92298013,   50.47656297,   59.66641187,
           0.        ,   80.95826449,    0.        ,   97.04482865,
          98.72729893],
       [  53.50968756,   54.14557214,    0.        , -100.        ,
        -100.        ,   61.77980767, -100.        ,   88.22035599,
         100.        ],
       [   0.        ,   52.50402036,   43.9359876 ,   51.09137525,
          61.00715483,   71.78642614,   73.94661407,   85.18458536,
          97.57257319],
       [   0.        ,   43.77254574, -1

In [345]:
# 3. Solve for optimal policy
def opt_policy(v):
    opt = []
    actions = ['W', 'N', 'E', 'S']
    
    # for all states
    for i in range(81):
        s = i+1    # value of state
    
        if v[i] != 0:
            # get possible next states given current state
            next_s_i = [i for i, x in enumerate(s_start) if x == s] 
            
            # use maximum value across all actions
            max_val = 0
            a_opt = 0
            for a in range(4):            
                # value iteration update
                val = r[i] + gamma*sum([p[a][j]*v[s_transitions[j,1] - 1] for j in next_s_i])
                
                # compare to current max value
                if val > max_val:
                    max_val = val
                    a_opt = actions[a]
            
            # update to maximum value
            opt.append(a_opt)
        else:
            opt.append('')
            
    return opt
    

In [347]:
np.transpose(np.array(opt_policy(v_curr)).reshape((9,9)))

array([['', '', '', '', '', '', '', '', ''],
       ['', 'E', 'E', 'E', 'S', 'S', '', '', 'W'],
       ['', 'N', '', 'E', 'E', 'E', 'E', 'E', 'N'],
       ['', 'N', 'E', 'N', '', 'N', '', 'E', 'S'],
       ['E', 'N', '', '', '', 'N', '', 'E', 'W'],
       ['', 'N', 'W', 'E', 'E', 'E', 'E', 'E', 'N'],
       ['', 'N', '', '', '', 'N', '', '', 'S'],
       ['', 'E', 'E', 'E', 'E', 'N', '', '', 'W'],
       ['', '', '', '', '', '', '', '', '']], dtype='<U1')